<a href="https://colab.research.google.com/github/TuckerRasbury/coding_sample_eviction-lab_data-engineer/blob/main/Eviction_Lab_Data_Engineer_Coding_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Sample - Data Pipeline for Evictions Filings
#### _Submission of Candidate Coding Sample for the role of Data Engineer at The Eviction Lab of Princeton University_


## Initial Prompt
---

In order to apply to the Data Engineer role at the Eviction Lab, I am spinning up a concise data pipeline to meet the fourth criteria laid out in the listing.

_"Applicants should submit a dossier including... (4) a coding sample or data product that speaks to applicant’s experience with relevant tasks"_

## Tasks Required of the Data Engineer
---

Here is an excerpt from the listing including what will be required of the Data Engineer for context.

_"The responsibilities of the position are to lead the development of a data construction pipeline for processing large-scale administrative records. This would involve writing code to create new data products (e.g., geocoding addresses, cleaning names, combining multiple sources of data) in a reproducible way; writing tests to assess the quality of the data products created by the pipeline; writing tests to assess the speed of the pipeline; optimizing the code to improve quality and speed; cleaning and reformatting incoming datasets to conform to the pipeline; running the pipeline using these datasets; and identifying and fixing bugs, among other tasks. The datasets used are very large and require the use of remote computing clusters. Applicants with experience using very large datasets and optimizing code to run efficiently are preferred."_

## Explanation of Script
---
In order to provide a coding sample to demonstrate some of the pre-requisite skills for this opening, herein I will spin up a light weight data pipeline. I would ideally like to gather more data, but in light of the U.S. Government Accountability Office's research  on eviction data availability being limited, I am going to leverage the csv download available datasets above [3].



### Part 1 - Ingesting CSV/Excel
For this part, I obtained data from the Legal Services Corporation (LSC) [2] and Zillow's publicly available datasets [3]. These datasets were downloadable as CSVs on their websites, stored to Github, and then ingested here. The method used below to ingest these datasets can easily be applied to Excel files as well.


#### Explaining the data collected
The first dataset from LSC is from their evictions tracker and according to them "provides access to multi-year trend data on eviction filings for 1,250 counties and municipalities in 30 states and territories across the United States." The second datasets from zillow represent their ZHVI and ZORI variables at the state and county levels. Those variables are explained below.

- Zillow Home Value Index (ZHVI): A measure of the typical home value and market changes across a given region and housing type.
- Zillow Observed Rent Index (ZORI): A smoothed measure of the typical observed market rate rent across a given region





### Part 2 - Ingesting from API
As part of this pipeline, I intended to ingest median household income data for all counties using the U.S. Census Bureau’s ACS 5-Year API [4](https://api.census.gov/data/2021/acs/acs5). While the API structure and variable targeting (e.g., B19013_001E for median income) were correctly implemented and previously functional, the Census API experienced extended availability issues during this project.

I was receving timeout error message so I tried to create some error handling with lags using the time library. I ultimately landed on using loops to retry my entry attempts. That failed as well, so to prevent this from blocking development, I went to the U.S. Census data site and downloaded county level median income data.


With the three data sources in hand and a short turn around time for pipeline development, I will proceed with the county level data I have across sources at the heart of the remaining work.





### Part 3: Aggregating Eviction Filings and Joining with Census Data

To connect eviction trends to socioeconomic data, I aggregated the Legal Services Corporation (LSC) weekly filings dataset to the **county-year level**. This aligns with the level of granularity of the American Community Survey (ACS), which is published annually.

Specifically, I:
- Grouped LSC data by `fips` and `year`, summing all eviction filings for each county-year combination
- Standardized the `fips` format to match ACS `county_fips` by padding to five characters
- Used DuckDB SQL to perform the join directly within Colab, demonstrating some SQL proficiency in a notebook environment

I joined this aggregated LSC dataset to ACS data on `county_fips` and `year`, adding into it:
- Median household income
- Total population
- Poverty rate

This enabled downstream analysis of how eviction volumes vary across income levels, states, and time — a critical step for equity-focused housing policy work.

The resulting dataset serves as a clean, interpretable foundation for further statistical modeling or visualization.



Data Sources and Appendix
---
1. Appendix - [Government Accountability Office - Evictions: National Data Are Limited and Challenging to Collect](https://www.gao.gov/products/gao-24-106637)

2. Data - [Civil Court Data Initiative. Legal Services Corporation, 2022.(accessed May 16, 2025)](https://civilcourtdata.lsc.gov/data/eviction)

3. Data - [Rental Data. Zillow. (accessed May 16, 2025)](https://www.zillow.com/research/data/)

4. Data - [U.S. Census Bureau’s ACS 5-Year API]((https://api.census.gov/data/2021/acs/acs5))



## Pre-Work

In [2]:
!pip install -q duckdb

## Establishing Libraries
import pandas as pd # used for data manipulation
import duckdb # data manipulation with SQL
import os
import requests # used for API Calls
import time # used for creating artificial delays to assist with data grabs
from google.colab import files # used to download tables
import statsmodels.api as sm # used for stats check of final table

## Part 1 - Ingesting CSV/Excel

### Ingesting - CSV - Zillow and Legal Services Corp (LSC)

In [3]:
# Importing Datasets

## Legal Services Corporation - Civil Court Data Initiative
### Weekly County Data
lsc_weekly_county_url = 'https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/weekly_county_data_download.csv'
lsc_weekly_county_df = pd.read_csv(lsc_weekly_county_url)

### Weekly State Data
lsc_weekly_state_url = 'https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/weekly_state_data_download.csv'
lsc_weekly_state_df = pd.read_csv(lsc_weekly_state_url)

# Add a small delay before the next request
time.sleep(2)

## Zillow House Value Data
## Zillow Home Value Index (ZHVI): A measure of the typical home value and market
## changes across a given region and housing type.


### Publicly Available Housing Data - County
zillow_county_url = 'https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
zillow_county_df = pd.read_csv(zillow_county_url)

### Publicly Available Housing Data - State
zillow_state_url = 'https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'
zillow_state_df = pd.read_csv(zillow_state_url)

# Add a small delay before the next request
time.sleep(2)

## Zillow Rental Price Data - County
## Zillow Observed Rent Index (ZORI): A smoothed measure of the typical observed
## market rate rent across a given region

### Publicly Available Rental Data - County
zillow_county_rental_url = 'https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/County_zori_uc_sfrcondomfr_sm_month.csv'
zillow_county_rental_df = pd.read_csv(zillow_state_url)

# Add a small delay before the next request
time.sleep(2)

In [4]:
# Example of designing tests to assess data shape (rows, columns)

print(" LSC shape:", lsc_weekly_county_df.shape)
print(" Zillow shape:", zillow_county_rental_df.shape)

 LSC shape: (252740, 4)
 Zillow shape: (51, 309)


In [5]:
# View Tables
lsc_weekly_county_df.head(20)

,fips,name,date,filings_count
0,33019,Sullivan,2022-01-03 12:00:00,4
1,33005,Cheshire,2022-02-07 12:00:00,3
2,33015,Rockingham,2022-03-21 12:00:00,14
3,33019,Sullivan,2022-03-28 12:00:00,2
4,33007,Coos,2022-04-04 12:00:00,2
5,33001,Belknap,2022-04-18 12:00:00,4
6,33013,Merrimack,2022-04-18 12:00:00,6
7,33019,Sullivan,2022-04-25 12:00:00,5
8,33017,Strafford,2022-05-02 12:00:00,6
9,33017,Strafford,2022-05-09 12:00:00,13


In [6]:
# View Tables - cont'd
zillow_county_rental_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28,2025-03-31,2025-04-30
0,9,0,California,state,NaN,193983.727686,194635.764895,195516.174793,197427.282169,199648.656602,...,794980.244270,796808.201343,799181.290966,800825.235948,802657.943456,804511.977573,804682.106782,803246.384103,799679.501223,796255.446598
1,54,1,Texas,state,NaN,114650.702784,114713.006937,114743.225427,114893.202475,114990.329964,...,311418.064885,310990.147334,310723.158782,310457.399594,310077.350638,309797.301597,309617.220803,309438.865180,308702.238205,307629.100564
2,14,2,Florida,state,NaN,108255.684033,108490.175922,108774.426833,109352.235824,109975.340556,...,400530.850389,399502.810192,398636.533116,397676.491442,396460.569464,395352.419957,394396.150784,393304.369817,391588.148457,389400.049453
3,43,3,New York,state,NaN,154607.546443,155158.101181,155688.098901,156827.979920,158007.349303,...,480811.117639,483958.227544,486867.601548,489325.354271,491186.473381,492969.583322,494116.872029,495314.625285,496232.692970,497620.899427
4,47,4,Pennsylvania,state,NaN,100690.506117,100905.875484,101108.264837,101520.703585,101944.381449,...,272989.988321,273246.797990,273644.011695,274407.296766,275318.143615,276537.984748,277632.128284,278567.608841,279045.436228,279450.903628


In [7]:
# Sidebar - Data Cleaning - LSC dataset

## Change the format of date to Month and Year columns without time
# Step 1 - Convert column to datetime
lsc_weekly_county_df['date'] = pd.to_datetime(lsc_weekly_county_df['date'])

# Step 2 - Extract year and month
lsc_weekly_county_df['year'] = lsc_weekly_county_df['date'].dt.year
lsc_weekly_county_df['month'] = lsc_weekly_county_df['date'].dt.month

# Step 3 - View new columns
lsc_weekly_county_df

,fips,name,date,filings_count,year,month
0,33019,Sullivan,2022-01-03 12:00:00,4,2022,1
1,33005,Cheshire,2022-02-07 12:00:00,3,2022,2
2,33015,Rockingham,2022-03-21 12:00:00,14,2022,3
3,33019,Sullivan,2022-03-28 12:00:00,2,2022,3
4,33007,Coos,2022-04-04 12:00:00,2,2022,4
...,...,...,...,...,...,...
252735,51033,Caroline,2018-04-09 12:00:00,1,2018,4
252736,18013,Brown,2023-11-13 12:00:00,1,2023,11
252737,18001,Adams,2024-09-02 12:00:00,3,2024,9
252738,38049,McHenry,2021-03-15 12:00:00,1,2021,3


## Part 2 - Ingesting - API - U.S. Census

In [8]:
# Create a working API Key

CENSUS_API_KEY = '86d117578634c16e49a8242b3a91ee1ee93e7834'

## For PROD, we would do this with some level of secret script seperately stored in the codebase.
## For this proof of concept, this is acceptable for now, but will need to be deleted/updated later.

In [9]:
# Create a list of U.S. State FIPS Codes

state_fips_list = [
    '01', '02', '04', '05', '06', '08', '09', '10', '11', '12',
    '13', '15', '16', '17', '18', '19', '20', '21', '22', '23',
    '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
    '34', '35', '36', '37', '38', '39', '40', '41', '42', '44',
    '45', '46', '47', '48', '49', '50', '51', '53', '54', '55',
    '56'
]

In [10]:
# Grab Median Income for All California Counties
## The data I'm targeting herein is the ACS (American Community Survey) 5-year estimates (poverty, rent burden, etc.)


# List of ACS 5-Year data years to include
acs_years = list(range(2010, 2024))


# Dictionary of variable codes and labels
acs_vars = {
    'B19013_001E': 'median_income',       # Median household income
    'B01003_001E': 'population_total',    # Total population
    'B17001_001E': 'poverty_universe',    # Poverty denominator
    'B17001_002E': 'poverty_count'        # Below poverty line count
}

# Build the variable string for the API
acs_var_string = ",".join(['NAME'] + list(acs_vars.keys()))

# Function to fetch ACS data for one state and one year
def fetch_acs_by_state_year(state_fips, year, retries=3, wait=10):
    url = f"https://api.census.gov/data/{year}/acs/acs5"
    params = {
        "get": acs_var_string,
        "for": "county:*",
        "in": f"state:{state_fips}",
        "key": CENSUS_API_KEY
    }

    for i in range(retries):
        try:
            print(f"Fetching state {state_fips}, year {year} (attempt {i+1})")
            r = requests.get(url, params=params, timeout=15)
            r.raise_for_status()
            data = r.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            df.rename(columns=acs_vars, inplace=True)
            df['median_income'] = pd.to_numeric(df['median_income'], errors='coerce')
            df['population_total'] = pd.to_numeric(df['population_total'], errors='coerce')
            df['poverty_count'] = pd.to_numeric(df['poverty_count'], errors='coerce')
            df['poverty_universe'] = pd.to_numeric(df['poverty_universe'], errors='coerce')
            df['poverty_rate'] = df['poverty_count'] / df['poverty_universe']
            df['county_fips'] = df['state'] + df['county']
            df['year'] = year
            return df
        except requests.exceptions.RequestException as e:
            print(f"Failed for state {state_fips}, year {year}: {e}")
            time.sleep(wait)

    print(f"Final failure for state {state_fips}, year {year}")
    return None


# Run the state loop
all_data = []

for year in acs_years:
    for state in state_fips_list:
        df = fetch_acs_by_state_year(state, year)
        if df is not None:
            all_data.append(df)

Fetching state 01, year 2010 (attempt 1)
Fetching state 02, year 2010 (attempt 1)
Fetching state 04, year 2010 (attempt 1)
Fetching state 05, year 2010 (attempt 1)
Fetching state 06, year 2010 (attempt 1)
Fetching state 08, year 2010 (attempt 1)
Fetching state 09, year 2010 (attempt 1)
Fetching state 10, year 2010 (attempt 1)
Fetching state 11, year 2010 (attempt 1)
Fetching state 12, year 2010 (attempt 1)
Fetching state 13, year 2010 (attempt 1)
Fetching state 15, year 2010 (attempt 1)
Fetching state 16, year 2010 (attempt 1)
Fetching state 17, year 2010 (attempt 1)
Fetching state 18, year 2010 (attempt 1)
Fetching state 19, year 2010 (attempt 1)
Fetching state 20, year 2010 (attempt 1)
Fetching state 21, year 2010 (attempt 1)
Fetching state 22, year 2010 (attempt 1)
Fetching state 23, year 2010 (attempt 1)
Fetching state 24, year 2010 (attempt 1)
Fetching state 25, year 2010 (attempt 1)
Fetching state 26, year 2010 (attempt 1)
Fetching state 27, year 2010 (attempt 1)
Fetching state 2

In [11]:
# Loop Through All State Year Combos

all_data = []

for year in acs_years:
    for state in state_fips_list:
        df = fetch_acs_by_state_year(state, year)
        if df is not None:
            all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Optional final cleanup
combined_acs_df = combined_df[[
    'county_fips', 'NAME', 'year',
    'median_income', 'population_total', 'poverty_count', 'poverty_universe', 'poverty_rate'
]]

print("Final shape:", combined_df.shape)
print(combined_acs_df.head())


Fetching state 01, year 2010 (attempt 1)
Fetching state 02, year 2010 (attempt 1)
Fetching state 04, year 2010 (attempt 1)
Fetching state 05, year 2010 (attempt 1)
Fetching state 06, year 2010 (attempt 1)
Fetching state 08, year 2010 (attempt 1)
Fetching state 09, year 2010 (attempt 1)
Fetching state 10, year 2010 (attempt 1)
Fetching state 11, year 2010 (attempt 1)
Fetching state 12, year 2010 (attempt 1)
Fetching state 13, year 2010 (attempt 1)
Fetching state 15, year 2010 (attempt 1)
Fetching state 16, year 2010 (attempt 1)
Fetching state 17, year 2010 (attempt 1)
Fetching state 18, year 2010 (attempt 1)
Fetching state 19, year 2010 (attempt 1)
Fetching state 20, year 2010 (attempt 1)
Fetching state 21, year 2010 (attempt 1)
Fetching state 22, year 2010 (attempt 1)
Fetching state 23, year 2010 (attempt 1)
Fetching state 24, year 2010 (attempt 1)
Fetching state 25, year 2010 (attempt 1)
Fetching state 26, year 2010 (attempt 1)
Fetching state 27, year 2010 (attempt 1)
Fetching state 2

#### Footnote

Given that the API solution is inconsistent and sometimes times out, I have gathered data from the U.S. Census bureau site, stored it on Github, and will read it in below similarly to how I did above to proceed with the pipeline to have on hand additionally. This can operate as a backup.

In [12]:
# Ingesting Hard Copies of Multiple Years of ACS

BASE_URL = "https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/"
YEARS = list(range(2010, 2024))  # 2010 to 2023

# Variable name for median income (this may vary slightly by year — adjust if needed)
INCOME_COLUMN = "S1901_C01_012E"

def load_acs_year(year):
    file_url = f"{BASE_URL}ACSST5Y{year}.S1901-Data.csv"
    print(f"Trying to load: {file_url}")
    try:
        df = pd.read_csv(file_url)

        # Rename income column if present
        if INCOME_COLUMN in df.columns:
            df = df.rename(columns={INCOME_COLUMN: "median_income"})
        else:
            print(f"Warning: Median income column '{INCOME_COLUMN}' not found in {year}")

        # Keep only relevant columns (you can expand this later)
        keep_cols = ['GEO_ID', 'NAME', 'median_income']
        df = df[[col for col in keep_cols if col in df.columns]]

        # Extract county_fips from GEO_ID (e.g., "0500000US06037" → "06037")
        df['county_fips'] = df['GEO_ID'].str.extract(r'US(\d{5})')

        # Add year
        df['year'] = year

        return df

    except Exception as e:
        print(f"Failed to load {year}: {e}")
        return None

# Load all years
dfs = [load_acs_year(y) for y in YEARS]
dfs = [df for df in dfs if df is not None]

# Combine
acs_all_years_df = pd.concat(dfs, ignore_index=True)

# Final cleaning
acs_all_years_df['median_income'] = pd.to_numeric(acs_all_years_df['median_income'], errors='coerce')
acs_all_years_df['county_fips'] = acs_all_years_df['county_fips'].astype(str).str.zfill(5)

# Preview
print("Combined shape:", acs_all_years_df.shape)
print(acs_all_years_df.head())

Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2010.S1901-Data.csv
Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2011.S1901-Data.csv
Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2012.S1901-Data.csv
Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2013.S1901-Data.csv
Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2014.S1901-Data.csv
Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2015.S1901-Data.csv
Trying to load: https://raw.githubusercontent.com/TuckerRasbury/coding_sample_eviction-lab_data-engineer/main/data/ACSST5Y2016.S1901-Data.csv
Trying

In [13]:
# View More of the Table
acs_all_years_df.head(20)

,GEO_ID,NAME,median_income,county_fips,year
0,Geography,Geographic Area Name,NaN,00nan,2010
1,0400000US36,New York,55603.0,00nan,2010
2,0500000US01001,"Autauga County, Alabama",53255.0,01001,2010
3,0500000US01003,"Baldwin County, Alabama",50147.0,01003,2010
4,0500000US01005,"Barbour County, Alabama",33219.0,01005,2010
5,0500000US01007,"Bibb County, Alabama",41770.0,01007,2010
6,0500000US01009,"Blount County, Alabama",45549.0,01009,2010
7,0500000US01011,"Bullock County, Alabama",31602.0,01011,2010
8,0500000US01013,"Butler County, Alabama",30659.0,01013,2010
9,0500000US01015,"Calhoun County, Alabama",38407.0,01015,2010


## Part 3 - PreWork - Review All Variables Available

In [16]:
# Zillow County Home Value Data
print("Zillow County - Home Value (zhvi):")
print(list(zillow_county_df.columns))  # Includes: RegionID, RegionName, State, Metro, SizeRank, Home Values, and monthly columns

# Zillow County Rental Estimates
print("nZillow County - Rent Estimates:")
print(list(zillow_county_rental_df.columns))  # Similar to above but focused on rent prices

# LSC Weekly County-Level Eviction Data
print("LSC Weekly County Data:")
print(list(lsc_weekly_county_df.columns))  # Expecting: state_fips, county_fips_code, week_start, filing_count, etc.

# ACS All Years (GitHub Hardcoded CSVs)
print("ACS Multi-Year (Static GitHub Data):")
print(list(acs_all_years_df.columns))  # Expecting: county_fips, NAME, year, median_income, etc.

# ACS All Years (Live Census API)
print("ACS Multi-Year (Live API Data):")
print(list(combined_acs_df.columns))  # Should match or extend acs_all_years_df, might include: poverty_rate, population_total


Zillow County - Home Value (zhvi):
['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS', '2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31', '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31', '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31', '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30', '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31', '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31', '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30', '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31', '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31', '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31', '2004-09-30', '2004-10-31', '2004-11-30', '2004-12-31', '20

### Tentative Join Key Alignment Across Datasets

| Dataset      | Possible Join Key |  Notes                                                                 |
|---------------------|-------------------|------------------------------------|
| Zillow Home Values and Rentals Data             | `RegionName + State`              | Requires external mapping to `county_fips` using a FIPS crosswalk     |
| Legal Services Corp Weekly Eviction Filings     | `state_fips + county_fips_code`   | Combine and zero-pad to create a valid 5-digit `county_fips` key      |
| American Community Survey (GitHub / API)  | `county_fips` + `year`            | Clean and directly joinable once date and FIPS are normalized         |


### Tentative Downstream Use Cases

| Dataset | Description |
|---------|-------------|
| **Zillow Home Values and Rentals Data** | Track monthly rent or home value changes; compare pricing trends across counties and time periods |
| **Legal Services Corp (LSC) Weekly Eviction Filing** | Analyze eviction filing rates over time; detect spikes related to policy or rental price shifts |
| **Census Bureua - American Community Survey (ACS) 5 Year Data** | Provide socioeconomic context (e.g., income, poverty rate, population) to support rate calculations and segmentation |
| **Combined** | Do eviction filings correlate with a county's average income? |
| **Combined** | Do eviction filings correlate with a (county's poverty count/poverty denominator)? |


## Part 3 - Joining the Data with SQL Using the Fips county Code

In [18]:
# Review the Datasets in Question - LSC
lsc_weekly_county_df.head(10000)

,fips,name,date,filings_count,year,month
0,33019,Sullivan,2022-01-03 12:00:00,4,2022,1
1,33005,Cheshire,2022-02-07 12:00:00,3,2022,2
2,33015,Rockingham,2022-03-21 12:00:00,14,2022,3
3,33019,Sullivan,2022-03-28 12:00:00,2,2022,3
4,33007,Coos,2022-04-04 12:00:00,2,2022,4
...,...,...,...,...,...,...
9995,72113,Ponce,2019-02-18 12:00:00,1,2019,2
9996,55005,Barron,2021-07-26 12:00:00,1,2021,7
9997,72057,Guayama,2024-07-08 12:00:00,1,2024,7
9998,55111,Sauk,2022-09-12 12:00:00,4,2022,9


In [19]:
# Review the Datasets in Question - ACS API
combined_acs_df.head(10000)

,county_fips,NAME,year,median_income,population_total,poverty_count,poverty_universe,poverty_rate
0,01001,"Autauga County, Alabama",2010,53255.0,53155,5623.0,53110.0,0.105875
1,01003,"Baldwin County, Alabama",2010,50147.0,175791,21216.0,173756.0,0.122102
2,01005,"Barbour County, Alabama",2010,33219.0,27699,6203.0,24809.0,0.250030
3,01007,"Bibb County, Alabama",2010,41770.0,22610,2830.0,22391.0,0.126390
4,01009,"Blount County, Alabama",2010,45549.0,56692,7510.0,56165.0,0.133713
...,...,...,...,...,...,...,...,...
9995,16053,"Jerome County, Idaho",2013,40126.0,22391,4056.0,22220.0,0.182538
9996,16055,"Kootenai County, Idaho",2013,49002.0,140785,18693.0,139113.0,0.134373
9997,16061,"Lewis County, Idaho",2013,36000.0,3851,677.0,3784.0,0.178911
9998,16067,"Minidoka County, Idaho",2013,43266.0,20104,2706.0,19940.0,0.135707


In [20]:
# Read dataframes into my SQL tool duckdb
duckdb.register('lsc_df', lsc_weekly_county_df)
duckdb.register('acs_df', combined_acs_df)

In [21]:
# Example of SQL -
# Leverage the fips codes at the county level to join the LSC and ACS datasets
# for later analysis

query = """
WITH lsc_yearly AS (
  SELECT
    LPAD(CAST(fips AS VARCHAR), 5, '0') AS county_fips,
    year,
    SUM(filings_count) AS total_filings
  FROM lsc_df
  GROUP BY county_fips, year
),

acs_cleaned AS (
  SELECT
    LPAD(CAST(county_fips AS VARCHAR), 5, '0') AS county_fips,
    year,
    NAME,
    median_income,
    population_total,
    poverty_rate
  FROM acs_df
)

SELECT
  l.year,
  l.county_fips,
  a.NAME AS name,
  l.total_filings AS annual_eviction_filings,
  a.median_income,
  a.population_total,
  a.poverty_rate
FROM lsc_yearly l
LEFT JOIN acs_cleaned a
  ON l.county_fips = a.county_fips AND l.year = a.year

"""

result_df = duckdb.query(query).to_df()

# View the result
enriched_county_fips_df = result_df
enriched_county_fips_df

,year,county_fips,name,annual_eviction_filings,median_income,population_total,poverty_rate
0,2022,33005,"Cheshire County, New Hampshire",458.0,76551.0,76610,0.090036
1,2022,33007,"Coos County, New Hampshire",238.0,55247.0,31430,0.116107
2,2022,33003,"Carroll County, New Hampshire",168.0,77049.0,50679,0.079997
3,2021,33007,"Coos County, New Hampshire",204.0,52054.0,31360,0.109364
4,2021,33003,"Carroll County, New Hampshire",154.0,70873.0,49961,0.078056
...,...,...,...,...,...,...,...
8485,2025,02150,None,1.0,NaN,<NA>,NaN
8486,2025,38055,None,1.0,NaN,<NA>,NaN
8487,2025,27029,None,1.0,NaN,<NA>,NaN
8488,2025,27087,None,1.0,NaN,<NA>,NaN


In [22]:
# View Columns Listed in Analytics Ready Table
print(list(enriched_county_fips_df.columns))

['year', 'county_fips', 'name', 'annual_eviction_filings', 'median_income', 'population_total', 'poverty_rate']


In [23]:
# Save the table for future use or delivery to client/stakeholder/project sprint
enriched_county_fips_df.to_csv("enriched_county_fips_df.csv", index=False)
files.download("enriched_county_fips_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# Running a linear regression using statsmodels to assess how median income and
# poverty rate relate to annual eviction filings per county

# Drop rows with missing values in the columns we will use for regression
regression_df = enriched_county_fips_df.dropna(subset=['annual_eviction_filings', 'median_income', 'poverty_rate'])

# Define dependent variable (y) and independent variables (X)
y = regression_df['annual_eviction_filings']
X = regression_df[['median_income', 'poverty_rate']]

# Add a constant to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the OLS (Ordinary Least Squares) model
model = sm.OLS(y, X)
results = model.fit()

# Print the regression summary
print(results.summary())


                               OLS Regression Results                              
Dep. Variable:     annual_eviction_filings   R-squared:                       0.022
Model:                                 OLS   Adj. R-squared:                  0.022
Method:                      Least Squares   F-statistic:                     74.59
Date:                     Tue, 20 May 2025   Prob (F-statistic):           9.34e-33
Time:                             06:27:13   Log-Likelihood:                -63780.
No. Observations:                     6564   AIC:                         1.276e+05
Df Residuals:                         6561   BIC:                         1.276e+05
Df Model:                                2                                         
Covariance Type:                 nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

### Interpretation of Regression Results

This regression model tested whether median income and poverty rate predict annual eviction filings at the county level.

Here's what I found:
- **Poverty rate is a strong, significant predictor** of higher eviction filing volumes. When the poverty rate in a county goes up by just a little bit (1 percentage point), the number of eviction filings goes up a lot — around 16,700 more filings across the dataset. That means counties with higher poverty tend to have more people getting eviction notices.
- **Median income showed a weaker and marginally significant positive relationship**, which may reflect noise or collinearity (ie. overlapping impact) with poverty rate.
- The overall model explains a small portion of the variance (R² = 0.022), meaning that it is likely that other unobserved factors not in this dataset (e.g., housing policy, court systems, rent burden) likely play major roles in eviction filings.

This statistical model illustrates how publicly available housing and economic data can be used to quantify and explore eviction risk.